In [ ]:
%%capture
! pip install -qqq wandb pytorch-lightning
! python -m spacy download fr

In [ ]:
import pytorch_lightning
# for some reason, need to run this twice

In [ ]:
import os
from pathlib import Path
import numpy as np
import math, copy, time



import torch
from torch import nn
from torch.utils.tensorboard import SummaryWriter
import torch.nn.functional as F
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader, random_split
from torchvision import transforms
from torch.autograd import Variable


# import pytorch_lightning as pl


In [ ]:
import pytorch_lightning as pl
pl.seed_everything(hash("set random seeds") % 2**32 - 1)


import wandb
from pytorch_lightning.loggers import WandbLogger

wandb.login()

<IPython.core.display.Javascript object>

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
# Test Text
# from torchtext import data, datasets
# import torchtext
from torchtext.data import Field, BucketIterator
# from torchtext.datasets import IMDB
from torchtext.datasets import IWSLT
# from torchtext.datasets import WikiText2 #vocab size of 33,278
import spacy


In [ ]:
# ! python -m spacy download fr

nlp = spacy.load("en")
doc = nlp("This is an English sentence.")
print([(w.text, w.pos_) for w in doc])

nlpf = spacy.load('fr')
docu = nlpf("Voici une phrase en francais.")
print([(w.text, w.pos_) for w in docu])



[('This', 'DET'), ('is', 'AUX'), ('an', 'DET'), ('English', 'ADJ'), ('sentence', 'NOUN'), ('.', 'PUNCT')]
[('Voici', 'VERB'), ('une', 'DET'), ('phrase', 'NOUN'), ('en', 'ADP'), ('francais', 'NOUN'), ('.', 'PUNCT')]


In [ ]:

PATH = './transformer_fr_en.pth'

DATA_PATH=Path('./data/')
DATA_PATH.mkdir(exist_ok=True)


BOS_TOKEN = '<s>'
EOS_TOKEN = '</s>'
PAD_TOKEN = "<pad>"
BLANK_TOKEN = "<blank>"
MAX_LEN = 100  # filter out examples that have more than MAX_LEN tokens
MIN_FREQ = 2

args = {
    "full_data_dir": DATA_PATH,
    "model_dimension" : 512,
    "num_layers" : 6,
    "num_heads" : 8,
#     "batch_size" : 4096, # batch size from the paper
    "batch_size" : 8,
    "dropout" : 0.1,
    "label_smoothing" : 0.1
}



In [ ]:
def greedy_decode_sequence(model, X, start_idx_Y, pad_idx_X, pad_idx_Y, max_len):
    '''
    Greedy Decoding
    X : a sequence of input symbols. shape(batch_size=b, inp_len)
    start_idx_Y: int.
    pad_idx_X: int.
    pad_idx_Y: int.
    max_len: int. Maximum context length for generation. Include <SOS> and <EOS>
    '''

    b, inp_len = X.shape
    inp_pads = (X == pad_idx_X).int()

    # encode inputs
    # shape (b, inp_len, d_model)
    encoded_memory = model.encode(X, inp_pads)
 
    # shape (b, max_len) e.g. (b, 10)
    Y = torch.ones(b, max_len).type_as(X) * pad_idx_Y
    Y[:,0] = start_idx_Y
    # shape (b, max_len)
    out_pads = (Y == pad_idx_Y).int()
  
    # generate one token at a time
    for t in range(1, max_len): # (1 to 9)

        # shape (b, t, d_model)
        decoder_output = model.decode(encoded_memory, Y[:, :t], inp_pads, out_pads[:, :t])

        # shape (b, t, V)
        decoded_logits = model.classifier(decoder_output)
        # decoded_probs = torch.softmax(decoded_logits, dim=-1)
    
        # shape (b,), (b,)
        max_prob, max_idx = torch.max(decoded_logits[:, -1, :], dim=-1)
    
        # update Y, out_pads for next timestep
        Y[:, t] = max_idx
        out_pads[:, t] = (max_idx == pad_idx_Y).int()

    # shape (b, max_len)
    # (1 to 9)
    return Y[:, 1:]

In [ ]:


spacy_en = spacy.load('en')
spacy_fr = spacy.load('fr')

def tokenize_french(text):
    return [tok.text for tok in spacy_fr.tokenizer(text)]

def tokenize_english(text):
    return [tok.text for tok in spacy_en.tokenizer(text)]

SRC = Field(tokenize=tokenize_french, pad_token=BLANK_TOKEN)
TGT = Field(tokenize=tokenize_english, init_token = BOS_TOKEN, eos_token = EOS_TOKEN, pad_token=BLANK_TOKEN)

MAX_LEN = 100
train, val, test = IWSLT.splits(
    exts=('.fr', '.en'), fields=(SRC, TGT), 
    filter_pred=lambda x: len(vars(x)['src']) <= MAX_LEN and len(vars(x)['trg']) <= MAX_LEN)
MIN_FREQ = 2
SRC.build_vocab(train.src, min_freq=MIN_FREQ)
TGT.build_vocab(train.trg, min_freq=MIN_FREQ)

SRC
print([tok.text for tok in spacy_fr.tokenizer("Je ne suis pas une malade.")])

downloading fr-en.tgz


fr-en.tgz: 100%|██████████| 25.7M/25.7M [00:07<00:00, 3.52MB/s]


.data/iwslt/fr-en/IWSLT16.TED.tst2013.fr-en.en.xml
.data/iwslt/fr-en/IWSLT16.TED.tst2010.fr-en.fr.xml
.data/iwslt/fr-en/IWSLT16.TED.tst2011.fr-en.fr.xml
.data/iwslt/fr-en/IWSLT16.TED.tst2014.fr-en.en.xml
.data/iwslt/fr-en/IWSLT16.TED.dev2010.fr-en.fr.xml
.data/iwslt/fr-en/IWSLT16.TED.tst2013.fr-en.fr.xml
.data/iwslt/fr-en/IWSLT16.TED.tst2014.fr-en.fr.xml
.data/iwslt/fr-en/IWSLT16.TED.tst2011.fr-en.en.xml
.data/iwslt/fr-en/IWSLT16.TED.tst2012.fr-en.fr.xml
.data/iwslt/fr-en/IWSLT16.TED.tst2012.fr-en.en.xml
.data/iwslt/fr-en/IWSLT16.TED.dev2010.fr-en.en.xml
.data/iwslt/fr-en/IWSLT16.TED.tst2010.fr-en.en.xml
.data/iwslt/fr-en/train.tags.fr-en.fr
.data/iwslt/fr-en/train.tags.fr-en.en
['Je', 'ne', 'suis', 'pas', 'une', 'malade', '.']


In [ ]:
type(train.src)

generator

In [ ]:
# atoi  -> ASCII to integer.
# atol  -> ASCII to long.
# atof  -> ASCII to floating.
# stoi  -> string to integer.
# stol  -> string to long.
# stoll -> string to long long.
# stof  -> string to float. 
# stod  -> string to double.
# stold -> string to long double.
# itos  -> integer to string

In [ ]:
print(SRC.vocab.stoi['dog'])
print(SRC.vocab.itos[666])
print(SRC.vocab.itos[33130])

print('\n')
print(TGT.vocab.stoi['canine'])
print(TGT.vocab.itos[666])
print(TGT.vocab.itos[27375])



33083
5
déambuler


27369
sitting
capitalization


In [ ]:
# Let's look at a batch of 4 sentences
train_iter = BucketIterator(train, batch_size=4, sort_key=lambda x: len(x.trg), shuffle=True)


In [ ]:

train_iter = BucketIterator(train, batch_size=4, sort_key=lambda x: len(x.trg), shuffle=True)

batch = next(iter(train_iter))
'''In each batch, the sentences have been transposed so they are descending vertically 
(important: we will need to transpose these again to work with the transformer). Each index represents a token (word), 
and each column represents a sentence. We have 10 columns, as 10 was the batch_size we specified.'''

print(batch.src) # source



In [ ]:
print(batch.trg) # target

In [ ]:
def save_cache(cache_path, dataset):
    with open(cache_path, 'w', encoding='utf-8') as cache_file:
        # Interleave source and target tokenized examples, source is on even lines, target is on odd lines
        for ex in dataset.examples:
            cache_file.write(' '.join(ex.src) + '\n')
            cache_file.write(' '.join(ex.trg) + '\n')

In [ ]:

def get_language_datasets(data_dir, max_len = MAX_LEN, min_freq = MIN_FREQ):

    spacy_en = spacy.load('en')
    spacy_fr = spacy.load('fr')

    def tokenize_fr(text):
        return [tok.text for tok in spacy_fr.tokenizer(text)]

    def tokenize_en(text):
        return [tok.text for tok in spacy_en.tokenizer(text)]

    
    # Tokenize for the source and target
    src_tokenizer = tokenize_en
    trg_tokenizer = tokenize_fr
    
    src_field_processor = Field(tokenize=src_tokenizer, pad_token=PAD_TOKEN, batch_first=True) # Whether to produce tensors with the batch dimension first. Default: False.
    trg_field_processor = Field(tokenize=trg_tokenizer, init_token=BOS_TOKEN, eos_token=EOS_TOKEN, pad_token=PAD_TOKEN, batch_first=True)

    fields = [('src', src_field_processor), ('trg', trg_field_processor)]

    # Only call once the splits function it is super slow as it constantly has to redo the tokenization
    prefix = 'en_fr_iwslt'
    
    train_cache_path = os.path.join(data_dir, f'{prefix}_train_cache.csv')
    val_cache_path = os.path.join(data_dir, f'{prefix}_val_cache.csv')
    test_cache_path = os.path.join(data_dir, f'{prefix}_test_cache.csv')

    # This simple caching mechanism gave me ~30x speedup on my machine! From ~70s -> ~2.5s!
    ts = time.time()
 
    src_ext = '.en' 
    trg_ext = '.fr' 
    
    train_dataset, val_dataset, test_dataset = IWSLT.splits(
        exts=(src_ext, trg_ext),
        fields=fields,
        root=data_dir,
        filter_pred=lambda x: len(x.src) <= max_len and len(x.trg) <= max_len
    )
    

    save_cache(train_cache_path, train_dataset)
    save_cache(val_cache_path, val_dataset)
    save_cache(test_cache_path, test_dataset)

#     print(f'Time it took to prepare the data: {time.time() - ts:3f} seconds.')
    print('It took {} seconds to prepare the data.'.format(time.time()-ts))

    # __getattr__ implementation in the base Dataset class enables us to call .src on Dataset objects even though
    # we only have a list of examples in the Dataset object and the example itself had .src attribute.
    # Implementation will yield examples and call .src/.trg attributes on them (and those contain tokenized lists)
    src_field_processor.build_vocab(train_dataset.src, min_freq=min_freq)
    trg_field_processor.build_vocab(train_dataset.trg, min_freq=min_freq)

    return train_dataset, val_dataset, src_field_processor, trg_field_processor


In [1]:
print(train_dataset[0])


class ENFRDataModule(pl.LightningDataModule):

    def __init__(self, data_dir='./', batch_size=128):
        super().__init__()
        self.data_dir = data_dir
        self.batch_size = batch_size
        self.transform = transforms.Compose([transforms.ToTensor()])
        
    def tokenize_fr(text):
        return [tok.text for tok in spacy_fr.tokenizer(text)]

    def tokenize_en(text):
        return [tok.text for tok in spacy_en.tokenizer(text)]


    def prepare_data(self):
        # download data, train then test

        spacy_en = spacy.load('en')
        spacy_fr = spacy.load('fr')


    def setup(self, stage=None):
      
        # Tokenize for the source and target
        src_tokenizer = tokenize_en
        trg_tokenizer = tokenize_fr
    
        src_field_processor = Field(tokenize=src_tokenizer, pad_token=PAD_TOKEN, batch_first=True) # Whether to produce tensors with the batch dimension first. Default: False.
        trg_field_processor = Field(tokenize=trg_tokenizer, init_token=BOS_TOKEN, eos_token=EOS_TOKEN, pad_token=PAD_TOKEN, batch_first=True)

        fields = [('src', src_field_processor), ('trg', trg_field_processor)]


        src_ext = '.en' 
        trg_ext = '.fr' 
        
        # we set up only relevant datasets when stage is specified
        if stage == 'fit' or stage is None:
           self.train_dataset, self.val_dataset, self.test_dataset = IWSLT.splits(
            exts=(src_ext, trg_ext),
            fields=fields,
            root=self.data_dir,
            filter_pred=lambda x: len(x.src) <= max_len and len(x.trg) <= max_len
        )
           src_field_processor.build_vocab(self.train_dataset.src, min_freq=min_freq)
           trg_field_processor.build_vocab(self.train_dataset.trg, min_freq=min_freq)

            # mnist = MNIST(self.data_dir, train=True, transform=self.transform)
            # self.mnist_train, self.mnist_val = random_split(mnist, [55000, 5000])
        if stage == 'test' or stage is None:
           self.train_dataset, self.val_dataset, self.test_dataset = IWSLT.splits(
            exts=(src_ext, trg_ext),
            fields=fields,
            root=self.data_dir,
            filter_pred=lambda x: len(x.src) <= max_len and len(x.trg) <= max_len
        )
           src_field_processor.build_vocab(self.train_dataset.src, min_freq=min_freq)
           trg_field_processor.build_vocab(self.train_dataset.trg, min_freq=min_freq)

            # self.mnist_test = MNIST(self.data_dir, train=False, transform=self.transform)

    # we define a separate DataLoader for each of train/val/test
    def train_dataloader(self):
        enfr_train = DataLoader(self.train_dataset, batch_size=self.batch_size)

        train_iter = BucketIterator(self.train_dataset, batch_size=self.batch_size, sort_key=lambda x: len(x.trg), shuffle=True)
        train_batch = next(iter(train_iter))
        # print(train_batch.src) 
        return train_batch

        # return enfr_train

    def val_dataloader(self):
        enfr_val = DataLoader(self.val_dataset, batch_size=10 * self.batch_size)

        val_iter = BucketIterator(self.val_dataset, batch_size=self.batch_size, sort_key=lambda x: len(x.trg), shuffle=True)
        val_batch = next(iter(val_iter))
        return val_batch
        # return enfr_val

    def test_dataloader(self):
        enfr_test = DataLoader(self.test_dataset, batch_size=10 * self.batch_size)
        test_iter = BucketIterator(self.train_dataset, batch_size=self.batch_size, sort_key=lambda x: len(x.trg), shuffle=True)
        test_batch = next(iter(test_iter))

        return test_batch
        # return enfr_test

# setup data
enfr = ENFRDataModule(data_dir = DATA_PATH)
enfr.setup()

# grab samples to log predictions on
samples = next(iter(enfr.val_dataloader()))

NameError: ignored

In [ ]:
# # How to get efficient batching

# '''While Torchtext is brilliant, it’s sort_key-based batching leaves 
# a little to be desired. Often the sentences are of different lengths, 
# and you end up feeding a lot of padding into your network 
# (as you can see with all the 1s in the last figure).

# Additionally, if your RAM can process say 1500 tokens each iteration, 
# and your batch_size is 20, then only when you have batches of length 75 
# utilising all the memory. 

# An efficient batching mechanism would change the batch size 
# depending on the sequence length to make sure around 1500 tokens were being processed each iteration.'''

global max_src_in_batch, max_tgt_in_batch

def batch_size_fn(new, count, sofar):
    "Keep augmenting batch and calculate total number of tokens + padding."
    global max_src_in_batch, max_tgt_in_batch
    if count == 1:
        max_src_in_batch = 0
        max_tgt_in_batch = 0
    max_src_in_batch = max(max_src_in_batch,  len(new.src))  #can change this .src to a more generic extension, #TODO: Review
    max_tgt_in_batch = max(max_tgt_in_batch,  len(new.trg) + 2)
    src_elements = count * max_src_in_batch
    tgt_elements = count * max_tgt_in_batch
    return max(src_elements, tgt_elements)


In [ ]:
# How to get masks and count the tokens in source and target sentences. Note that 

def masks_tokens(src_token_ids_batch, trg_token_ids_batch, pad_token_id, device):
    
    def masks_tokens_src(src_token_ids_batch, pad_token_id):
        batch_size = src_token_ids_batch.shape[0]

        # src_mask shape = (B, 1, 1, S) #TODO: check this
        # src_mask only masks pad tokens as we want to ignore their representations (no information in there...)
        src_mask = (src_token_ids_batch != pad_token_id).view(batch_size, 1, 1, -1)
        num_src_tokens = torch.sum(src_mask.long())

        return src_mask, num_src_tokens



    def masks_tokens_trg(trg_token_ids_batch, pad_token_id):
        batch_size = trg_token_ids_batch.shape[0]
        device = trg_token_ids_batch.device

        # Same as src_mask but we additionally want to mask future tokens (want to predict)
        # Note: wherever the mask value is true we want to attend to that token, otherwise we mask (ignore) it.
        sequence_length = trg_token_ids_batch.shape[1]  # trg_token_ids shape = (B, T) where T max trg token-sequence length
        trg_padding_mask = (trg_token_ids_batch != pad_token_id).view(batch_size, 1, 1, -1)  # shape = (B, 1, 1, T)
        trg_no_look_forward_mask = torch.triu(torch.ones((1, 1, sequence_length, sequence_length), device=device) == 1).transpose(2, 3)

        # logic AND operation (both padding mask and no-look-forward must be true to attend to a certain target token)
        trg_mask = trg_padding_mask & trg_no_look_forward_mask  # final shape = (B, 1, T, T)
        num_trg_tokens = torch.sum(trg_padding_mask.long())

        return trg_mask, num_trg_tokens

    src_mask, num_src_tokens = masks_tokens_src(src_token_ids_batch, pad_token_id)
    trg_mask, num_trg_tokens = masks_tokens_trg(trg_token_ids_batch, pad_token_id)

    return src_mask, trg_mask, num_src_tokens, num_trg_tokens

In [ ]:
# Training Loop: Ignore
for batch_idx, token_ids_batch in enumerate(token_ids_loader):
            src_token_ids_batch, trg_token_ids_batch_input, trg_token_ids_batch_gt = get_src_and_trg_batches(token_ids_batch)
            src_mask, trg_mask, num_src_tokens, num_trg_tokens = get_masks_and_count_tokens(src_token_ids_batch, trg_token_ids_batch_input, pad_token_id, device)

            # log because the KL loss expects log probabilities (just an implementation detail)
            predicted_log_distributions = baseline_transformer(src_token_ids_batch, trg_token_ids_batch_input, src_mask, trg_mask)
            smooth_target_distributions = label_smoothing(trg_token_ids_batch_gt)  # these are regular probabilities

            if is_train:
                custom_lr_optimizer.zero_grad()  # clean the trainable weights gradients in the computational graph

            loss = kl_div_loss(predicted_log_distributions, smooth_target_distributions)

            if is_train:
                loss.backward()  # compute the gradients for every trainable weight in the computational graph
                custom_lr_optimizer.step()  # apply the gradients to weights


In [ ]:
# View some of our samples
def custom_dataloader(data_dir, batch_size, device):
    train_dataset, val_dataset, src_field_processor, trg_field_processor = get_language_datasets(data_dir=DATA_PATH)
    
    '''batch_size_fn: 
    Function of three arguments (new example to add, current count of examples in the batch, and current effective batch size)
            that returns the new effective batch size resulting from adding
            that example to a batch. This is useful for dynamic batching, where
            this function would add to the current effective batch size the
            number of tokens in the new example.'''

    # using default sorting function which
    train_token_ids_loader, val_token_ids_loader = BucketIterator.splits(
     datasets=(train_dataset, val_dataset), batch_size=batch_size,
     device=device,
     sort_within_batch=True,
     batch_size_fn=batch_size_fn
    )

    return train_token_ids_loader, val_token_ids_loader, src_field_processor, trg_field_processor



In [ ]:
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device("cpu")
device

device(type='cpu')

In [ ]:
# Load the data

train_token_ids_loader, val_token_ids_loader, src_field_processor, trg_field_processor = custom_dataloader(data_dir=DATA_PATH, 
                                                                                                           batch_size=8, device=device)

# These are generator objects


downloading en-fr.tgz


en-fr.tgz: 100%|██████████| 26.3M/26.3M [00:07<00:00, 3.59MB/s]


data/iwslt/en-fr/IWSLT16.TED.tst2010.en-fr.fr.xml
data/iwslt/en-fr/IWSLT16.TED.tst2011.en-fr.en.xml
data/iwslt/en-fr/IWSLT16.TED.tst2013.en-fr.fr.xml
data/iwslt/en-fr/IWSLT16.TED.tst2012.en-fr.en.xml
data/iwslt/en-fr/IWSLT16.TED.tst2011.en-fr.fr.xml
data/iwslt/en-fr/IWSLT16.TED.tst2013.en-fr.en.xml
data/iwslt/en-fr/IWSLT16.TED.tst2014.en-fr.fr.xml
data/iwslt/en-fr/IWSLT16.TED.tst2012.en-fr.fr.xml
data/iwslt/en-fr/IWSLT16.TED.dev2010.en-fr.fr.xml
data/iwslt/en-fr/IWSLT16.TED.tst2010.en-fr.en.xml
data/iwslt/en-fr/IWSLT16.TED.tst2014.en-fr.en.xml
data/iwslt/en-fr/IWSLT16.TED.dev2010.en-fr.en.xml
data/iwslt/en-fr/train.tags.en-fr.fr
data/iwslt/en-fr/train.tags.en-fr.en
It took 73.89170169830322 seconds to prepare the data.


In [ ]:
pad_token_id = src_field_processor.vocab.stoi[PAD_TOKEN]
print(pad_token_id)

for batch in train_token_ids_loader:
    # Visually inspect that masks make sense
    src_padding_mask, trg_mask, num_src_tokens, num_trg_tokens = masks_tokens(batch.src, batch.trg, pad_token_id, device)
    break

# Check vocab size
print('The source vocabulary length: {}.'.format(len(src_field_processor.vocab)))
print('The target vocabulary length: {}.'.format(len(trg_field_processor.vocab)))





1


ValueError: ignored

In [ ]:
# helper function to inspect the text
def sample_text_from_loader(src_field_processor, trg_field_processor, token_ids_loader, num_samples=2, sample_src=True, sample_trg=True, show_padded=False):
    
    assert sample_src or sample_trg, f'Either src or trg or both must be enabled.'

    for b_idx, token_ids_batch in enumerate(token_ids_loader):
        if b_idx == num_samples:  # Number of sentence samples to print
            break

        print('-' * 10)
        if sample_src:
            print("Source text:", end="\t")
            for token_id in token_ids_batch.src[0]:  # print only the first example from the batch
                src_token = src_field_processor.vocab.itos[token_id]

                if src_token == PAD_TOKEN and not show_padded:
                    continue

                print(src_token, end=" ")
            print()

        if sample_trg:
            print("Target text:", end="\t")
            for token_id in token_ids_batch.trg[0]:
                trg_token = trg_field_processor.vocab.itos[token_id]

                if trg_token == PAD_TOKEN and not show_padded:
                    continue

                print(trg_token, end=" ")
            print("\n")

sample_text_from_loader(src_field_processor, trg_field_processor, train_token_ids_loader, num_samples=5)


In [ ]:
#jachiam


B = 5
L = 32
d = 16
w = 8

x = torch.as_tensor(np.random.rand(B,L,d), dtype=torch.float32)

class Attention(nn.Module):

    def __init__(self, d_in = 16, d_proj = 8, n_heads = 2, causal = True):  #ask josh why causal here  
        #a causal convolution has a window that overlaps only the current and previous timesteps
        #Causal convolutions are necessary because it would be cheating if the CNN was able to 
        #“see” information from the future timesteps that it is trying to predict.
        super().__init__()
        self.d_proj = d_proj
        self.n_heads = n_heads
        self.qkv_lin = nn.Linear(d_in, 3 * d_proj * n_heads, bias=False)
        self.causal = True

    def _mask(self, qk):
        mask = 1e9 * (torch.tril(torch.ones_like(qk)) - 1)
        return torch.tril(qk) + mask

    def forward(self, x):
        qkv_lin = self.qkv_lin(x).reshape(*x.shape[:2], self.n_heads, self.d_proj * 3) # B x L x n x 3d
        qkv_lin = qkv_lin.permute(0,2,1,3)                  # B x n x L x 3d
        q, k, v = torch.split(qkv_lin, self.d_proj, dim=-1) # B x n x L x d
        q_pre = q.reshape(-1, *q.shape[2:])                 # Bn x L x d
        k_pre = k.reshape(-1, *k.shape[2:]).permute(0,2,1)  # Bn x d x L
        v_pre = v.reshape(-1, *v.shape[2:])                 # Bn x L x d
        qk = torch.bmm(q_pre,k_pre)                         # Bn x L x L
        qk_ = self._mask(qk) if self.causal else qk
        softmax_qk = torch.softmax(qk_, axis=-1)
        
        y = torch.bmm(softmax_qk, v_pre)                    # Bn x L x d
        y = y.reshape(x.shape[0], self.n_heads, x.shape[1], self.d_proj)  # B x n x L x d
        y = y.permute(0,2,1,3)
        y = y.reshape(*x.shape[:2],-1)
        return y, softmax_qk.reshape(x.shape[0], self.n_heads, x.shape[1], x.shape[1])
        #q, k, v = torch.split(qkv_lin, self.d_proj, dim=2)
        #return q, k, v

In [ ]:
embedding = nn.Embedding(10, 3) # an Embedding module containing 10 tensors of size 3
embedding

input = torch.LongTensor([[1,2,4],[4,3,2], [2,3,1]])
input2 = torch.LongTensor([[0,8], [7,6]])
print(input)
print(embedding(input))

print(input2)
print(embedding(input2))


In [ ]:
# Input : Embeddings + Positional Encodings
src_vocab_size = len(src_field_processor.vocab)
trg_vocab_size = len(trg_field_processor.vocab)
# Embeddings
class Embedding(nn.Module):
    def __init__(self, src_vocab_size, trg_vocab_size):
        super().__init__()
#         self.embeddings_lookup = nn.Embedding(length,  embedding_dim = 512)
        self.source_embedding = nn.Embedding(num_embeddings=src_vocab_size, embedding_dim = 512)
        self.target_embedding = nn.Embedding(num_embeddings=trg_vocab_size, embedding_dim = 512)
        
#     def forward(x, token_ids_batch):
        
        
        

# Positional Encoding
# class PositionalEncoding(nn.Module):
    

In [ ]:

embedding = nn.Embedding(1000,512)
embedding(torch.LongTensor([3,4]))

In [ ]:
# Inspect embedding architecture
emb = nn.Embedding(num_embeddings=src_vocab_size, embedding_dim = 512)


# batch_id, batch = enumerate(train_token_ids_loader)
for b_idx, token_ids_batch in enumerate(train_token_ids_loader):
    print(b_idx)
#     print(token_ids_batch)

    if b_idx >= 2: 
        break
        
    
    if b_idx == 0:
        for token_id in token_ids_batch.src[0]:  
            print("The token")
            print(token_id)
            print(token_ids_batch.src[0])
    #         print(token_ids_batch.src[b_idx])
            print("Shape of token: {}".format((token_ids_batch.src[0].shape)))
            print("The Embedding")
            print(emb(token_ids_batch.src[0]))   #need CPU device
            print("Size of Embedding: {}".format(emb(token_ids_batch.src[0]).shape))


        

In [ ]:
d_model = 512 # model dimension
# for b_idx, token_ids_batch in enumerate(train_token_ids_loader):
#     if b_idx < 2:
#         print(token_ids_batch)
#         print(token_ids_batch.src.ndim)

class InputEmbedding(nn.Module):
    def __init__(self, dataloader, vocab_size):
        super().__init__()
        self.embeddings_lookup = nn.Embedding(num_embeddings = vocab_size, embedding_dim = 512)
        
        
    def forward(self, token_ids_batch):
        return(self.embeddings_lookup(token_ids_batch)*math.sqrt(512))
    

wow = InputEmbedding(dataloader = train_token_ids_loader, vocab_size = src_vocab_size)

for b_idx, token_ids_batch in enumerate(train_token_ids_loader):
    if b_idx >= 1:
        break
        
    if b_idx < 1:
        for token_id in token_ids_batch.src[0]:  
            print(wow.forward(token_ids_batch.src))
            print(wow.forward(token_ids_batch.trg))


In [ ]:
# PE(pos, 2i)= sin(pos/(10000)^(2i/d_model))
# PE(pos, 2i+1)= cos(pos/(10000)^(2i/d_model))

# max possible sequence length here is 100, sampled input and output sequences smaller than 100, but we might want a buffer so set to 150

# position_id = torch.arange(0, 150).unsqueeze(1)
# print(position_id)

# inter = pos/(np.power(10000, (2 * (i//2)) / np.float32(d_model)))

# i=0
# expo = torch.pow(10000., 2*i/512)
# print(expo)

class PositionalEncoding(nn.Module):
    def __init__(self, position, d_model):
        super().__init__()
        self.d_model = 512
        self.dropout = nn.Dropout(p= 0.1)
        
        def get_inter(pos, i, d_model):
            return pos* (1 / np.power(10000, (2 * (i//2)) / np.float32(d_model)))

        self.wave_pos = get_inter(torch.arange(position).unsqueeze(1), torch.arange(d_model).unsqueeze(1), d_model = 512)
#         self.wave_pos = get_inter(np.arange(position)[:, np.newaxis],  np.arange(d_model)[np.newaxis, :],  d_model = 512)
        
        # apply sin to even indices in the array; 2i
        wave_pos[:, 0::2] = np.sin(wave_pos[:, 0::2])
  
        # apply cos to odd indices in the array; 2i+1
        wave_pos[:, 1::2] = np.cos(wave_pos[:, 1::2])
        
        
        
    def forward(x, embeddings):
        return self.dropout(wave_pos + embeddings)
        
        


In [ ]:
tokens = 10
dimensions = 64
   
# PE(pos, 2i)= sin(pos/((10000)^(2i/d_model)))
# PE(pos, 2i+1)= cos(pos/((10000)^(2i/d_model)))

    
class InputEmbeddings(nn.Module):
    def __init__(self, d_model, vocab):
        super(InputEmbeddings, self).__init__()
        self.tokens = nn.Embedding(vocab, d_model)
        self.d_model = d_model

    def forward(self, x):
        return self.tokens(x) * math.sqrt(self.d_model)

    
class PositionalEncoding(nn.Module):
    "Implement the PE function."
    def __init__(self, d_model, dropout, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=0.1)
        
        # Compute the positional encodings once in log space.
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len).unsqueeze(1) # position indexes 0-4999
#         div_term = torch.exp(torch.arange(0, d_model, 2) *  -(math.log(10000.0) / d_model)) # 10000 = 2 * max_len
        div_term = torch.pow(10000., -torch.arange(0, d_model, 2, dtype=torch.float)/d_model )

        
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)
        
    def forward(self, x):
        x = x + Variable(self.pe[:, :x.size(1)], requires_grad=False)
        return self.dropout(x)
    

In [ ]:
# So we pass the token ids through the embedding and positional encoding (separately or sequentially). I want to do it separately
pure_emb = InputEmbeddings( vocab = src_vocab_size, d_model = 512)

pos_enc = PositionalEncoding(d_model = 512, dropout=0.1, max_len=5000)
d_matrix = 64
n_heads = 8
d_model = 512



lin_layers = nn.Linear(in_features=d_model, out_features= 3 * d_matrix * n_heads, bias=False) #input: Batch_size X n_tokens X d_model


print(lin_layers)

for b_idx, token_ids_batch in enumerate(train_token_ids_loader):
    if b_idx >= 1:
        break
        
        
    if b_idx < 1:
        first = 0
        for token_id in token_ids_batch.src[0]: 
            if first == 0:
                print("Source sentence embedding")
                print(pure_emb.forward(token_ids_batch.src))
                print("Source sentence embedding plus positional encoding and dropout")
                print(pos_enc.forward(pure_emb.forward(token_ids_batch.src)))
                print(pos_enc.forward(pure_emb.forward(token_ids_batch.src)).shape)
                first += 1

            

# Sweet
# Now, self-attention






Linear(in_features=512, out_features=1536, bias=False)
Source sentence embedding
tensor([[[ 26.2299, -24.1689,  36.3473,  ..., -19.3473,   8.8194,   0.7304],
         [-32.2750,  11.1693, -18.6711,  ..., -47.1638,  -3.1729,  15.1690],
         [ 22.2667,  17.1060, -22.6408,  ...,  52.9551,  59.0051,  13.5322],
         ...,
         [ 36.6176,  52.9275,  -1.8158,  ..., -28.0755,  38.5611,  11.6662],
         [ -8.5879,  -6.3386,  -2.1202,  ..., -35.4681,   2.0181,  14.2503],
         [-16.4602,   3.1160,  -0.1835,  ...,  -5.7261, -26.7490,  14.4420]]],
       grad_fn=<MulBackward0>)
Source sentence embedding plus positional encoding and dropout
tensor([[[ 29.1443, -25.7432,  40.3859,  ..., -20.3859,   0.0000,   1.9227],
         [ -0.0000,  13.0106, -19.8325,  ..., -51.2932,  -0.0000,  17.9656],
         [  0.0000,  18.5443, -24.1160,  ...,  59.9502,  65.5614,  16.1469],
         ...,
         [  0.0000,  58.5026,  -2.7260,  ...,  -0.0000,   0.0000,  14.0735],
         [-10.3765,  -6.3

In [ ]:
def clones(module, n_copies):
    "Produce N identical layers."
    return nn.ModuleList([copy.deepcopy(module) for _ in range(n_copies)])



0
1
2
3


In [ ]:
class Attention1(nn.Module):
    def __init__(self, n_heads, d_model, dropout):
        super(Attention1, self).__init__()
        # take the token embedding and multiply by W matrices (512 x (512/number of heads))
        assert d_model%n_heads == 0, f'Model dimension must be divisible by the number of heads.'
        d_matrix = int(d_model/n_heads)
        
#         self.init = nn.init.xavier_uniform_(m.weight)
        self.heads = n_heads
        self.dropout = nn.Dropout(p=dropout)
        self.qkv_lin = nn.Linear(in_features=d_model, out_features= 3 * d_matrix * n_heads, bias=False) 
        #input of qkv linear: Batch_size X L_tokens X d_model
        # output of qkv linear: Batch_size x L_tokens x d_model*3
           
        
#         self.linears = clones(nn.Linear(in_features=d_model, out_features=d_model), n_copies=4)     
#         W_Q = torch.rand(d_model, d_matrix)
        
    def forward(self, x): 
        return(self.qkv_lin(x))
        


        

In [ ]:
# So we pass the token ids through the embedding and positional encoding (separately or sequentially). I want to do it separately
pure_emb = InputEmbeddings( vocab = src_vocab_size, d_model = 512)

pos_enc = PositionalEncoding(d_model = 512, dropout=0.1, max_len=5000)
d_matrix = 64
n_heads = 8
d_model = 512



att = Attention1(n_heads = 8, d_model = 512, dropout=0.1)

print(lin_layers)

for b_idx, token_ids_batch in enumerate(train_token_ids_loader):
    if b_idx >= 1:
        break
        
        
    if b_idx < 1:
        first = 0
        for token_id in token_ids_batch.src[0]: 
            if first == 0:
                print("Source sentence embedding")
                EMB = pure_emb.forward(token_ids_batch.src)
                print(EMB)
                print("Source sentence embedding plus positional encoding and dropout")
                POS = pos_enc.forward(EMB)
                print(POS)
                print(POS.shape)
                print("Linear Layer Passthrough")
                QKV_LIN = lin_layers.forward(POS)
#                 print(QKV_LIN)
                print(QKV_LIN.shape)
                print("Attempt at reshaping, so we pop the 3*d_model into a different dimension")
#                 print(QKV_LIN.shape[:2], n_heads, d_matrix)
#                 print(QKV_LIN.reshape(*QKV_LIN.shape[:2], n_heads, d_matrix*3))
                print(QKV_LIN.reshape(*QKV_LIN.shape[:2], n_heads, d_matrix*3).shape)
    
                QKV_RESHAPED = QKV_LIN.reshape(QKV_LIN.shape[0], QKV_LIN.shape[1], n_heads, d_matrix*3)
#                 print(QKV_RESHAPED.shape)
                print("First we change the shape in this way")
                QKV_PERMUTED = QKV_LIN.reshape(QKV_LIN.shape[0], n_heads, QKV_LIN.shape[1], d_matrix*3)
                print(QKV_PERMUTED.shape)
                
                print("So now we have the reshaped tensor, we have our heads, each with QKV joined like [QKV_1, QKV_2 ... , QKV_n_heads]")
                print("Now we want to split the matrices into W_Q, W_K, W_V, in that order")
                Q, K, V = torch.split(QKV_PERMUTED, split_size_or_sections = d_matrix, dim=-1) # B x n x L x d
                print(Q.shape)
#                 q_pre = Q.reshape(-1, Q.shape[0], Q.shape[1]) # equivalent to Q.reshape(Q.shape[2]*Q.shape[3], Q.shape[0], Q.shape[1])
#  q.reshape(-1, *q.shape[2:]) 
                
                q_pre2 = Q.reshape(Q.shape[0]*Q.shape[1], Q.shape[2], Q.shape[3])
                k_pre2 = K.reshape(K.shape[0]*K.shape[1], K.shape[3], K.shape[2]) # note that the last 2 dimensions are transposed here for multiplication
                v_pre2 = V.reshape(V.shape[0]*V.shape[1], V.shape[2], V.shape[3])
                
                print("Shape before mult")
#                 print(q_pre.shape)
                print(q_pre2.shape)
                print(k_pre2.shape)
#                 print((q_pre2*k_pre2).shape)
#                 print(torch.bmm(q_pre2, k_pre2).shape)
#                 print(torch.matmul(q_pre2, k_pre2).shape)
        
                print("Now QK_soft")
                QK = torch.matmul(q_pre2, k_pre2)/math.sqrt(d_matrix)
                QK_soft = F.softmax(QK, dim = -1)  # dim : A dimension along which Softmax will be computed (so every slice along dim will sum to 1).


                print(QK_soft.shape)
                print("Final values")
                
                final_val = torch.matmul(QK_soft,v_pre2)
        
#                 y = y.reshape(x.shape[0], self.n_heads, x.shape[1], self.d_proj)
                print(final_val.shape)
                y = final_val.reshape(POS.shape[0], n_heads, POS.shape[1], d_matrix)
                y = y.permute(0,2,1,3)   # B x L x n x d
                y = y.reshape(*POS.shape[:2],-1)
                print("reshaped final")
#                 print(y)
                
                x = final_val.reshape(POS.shape[0], POS.shape[1], n_heads, d_matrix)
                x = x.reshape(POS.shape[0], POS.shape[1], -1)
                print("new test")
#                 print(x)
             
                first += 1

            



Linear(in_features=512, out_features=1536, bias=False)
Source sentence embedding
tensor([[[ 2.6072e+01, -1.8959e+01, -2.1574e+01,  ..., -1.0317e+01,
          -2.5158e+01, -2.2811e+01],
         [ 1.8705e+01,  9.5811e+00,  2.2174e+01,  ...,  1.4913e+00,
           4.0880e-02,  1.4107e+01],
         [-2.0561e+01,  1.7480e+00,  1.7252e+01,  ...,  2.5553e+00,
          -1.9070e+01, -8.2803e+00],
         ...,
         [-4.2242e+00,  2.3923e+00,  4.3365e+01,  ..., -9.9742e+00,
           3.2115e+01,  4.0188e+01],
         [-2.7822e+01,  2.6559e+01,  3.8341e+01,  ...,  3.9539e+01,
          -5.4620e+00,  1.7267e+01],
         [ 3.1210e+01,  1.0616e+01, -4.8651e-01,  ...,  2.7768e+01,
           2.5556e+00,  5.9681e+00]]], grad_fn=<MulBackward0>)
Source sentence embedding plus positional encoding and dropout
tensor([[[ 2.8969e+01, -1.9954e+01, -2.3972e+01,  ..., -1.0352e+01,
          -2.7953e+01, -2.4234e+01],
         [ 0.0000e+00,  1.1246e+01,  0.0000e+00,  ...,  0.0000e+00,
           4.

In [ ]:

# Now, self-attention

class Attention2(nn.Module):
    def __init__(self, n_heads, d_model, dropout):
        super(Attention2, self).__init__()
        # take the token embedding and multiply by W matrices (512 x (512/number of heads))
        assert d_model%n_heads == 0, 'Model dimension must be divisible by the number of heads.'
        d_matrix = int(d_model/n_heads)
        self.d_matrix = d_matrix
        
        # self.init = nn.init.xavier_uniform_(m.weight)
        self.n_heads = n_heads
        self.dropout = nn.Dropout(p=dropout)
        self.qkv_lin = nn.Linear(in_features=d_model, out_features= 3 * self.d_matrix * self.n_heads, bias=False) 
        
        #input of qkv linear: Batch_size X n_tokens X d_model
        # output of qkv linear: Batch_size x n_tokens x d_model*3
        
    def forward(self, x):
        QKV_LIN = self.qkv_lin(x).reshape(*x.shape[:2], self.n_heads, self.d_matrix*3)
#         QKV_LIN = self.qkv_lin(x).reshape(x.shape[0], self.n_heads, x.shape[1], self.d_matrix * 3)
        
        QKV_PERMUTED = QKV_LIN.reshape(QKV_LIN.shape[0], self.n_heads, QKV_LIN.shape[1], self.d_matrix*3)
        Q, K, V = torch.split(QKV_PERMUTED, split_size_or_sections = self.d_matrix, dim=-1)
        
        q_prep = Q.reshape(Q.shape[0]*Q.shape[1], Q.shape[2], Q.shape[3])
        k_prep = K.reshape(K.shape[0]*K.shape[1], K.shape[3], K.shape[2])
        v_prep = V.reshape(V.shape[0]*V.shape[1], V.shape[2], V.shape[3])
        
        QK = torch.matmul(q_prep, k_prep)/math.sqrt(self.d_matrix)
        QK_soft = self.dropout(F.softmax(QK, dim = -1))  # dim : A dimension along which Softmax will be computed (so every slice along dim will sum to 1).
        # Masks and Dropout
        QKV = torch.matmul(QK_soft,v_prep)
        
#         out = QKV.reshape(x.shape[0], self.n_heads, x.shape[1], self.d_matrix)
        out = QKV.reshape(x.shape[0], x.shape[1], self.n_heads, self.d_matrix)
        out = out.reshape(x.shape[0], x.shape[1], -1)
        
        return out, QK_soft.reshape(x.shape[0], self.n_heads, x.shape[1], x.shape[1])
        
    
#      1 x 8 x 23 x 192
#      1 x 8 x 23 x 64
#          8 x 23 x 64
#          8 x 64 x 23
#       n_heads x L x L
#       n_heads x L x 64


In [ ]:
class FFNN(nn.Module):
    def __init__(self, d_in, d_within, dropout=0.1):
        super(FFNN, self).__init__()
        self.linear_in = nn.Linear(d_in, d_within)
        self.linear_out = nn.Linear(d_within, d_in)
        self.dropout = nn.Dropout(p=dropout)
        
    def forward(self, x):
        out = self.linear_out(self.dropout(F.relu(self.linear_in(x))))
        return out

In [ ]:
pure_emb = InputEmbeddings( vocab = src_vocab_size, d_model = 512)

pos_enc = PositionalEncoding(d_model = 512, dropout=0.1, max_len=5000)
d_matrix = 64
n_heads = 8
d_model = 512

att = Attention2(n_heads = 8, d_model = 512, dropout=0.1)
ffnn = FFNN(d_in = d_model, d_within=2048)

print(lin_layers)

for b_idx, token_ids_batch in enumerate(train_token_ids_loader):
    if b_idx >= 1:
        break
        
    if b_idx < 1:
        first = 0
        for token_id in token_ids_batch.src[0]: 
            if first == 0:
                print("Source sentence embedding")
                EMB = pure_emb.forward(token_ids_batch.src)
                print(EMB)
                print("Source sentence embedding plus positional encoding and dropout")
                POS = pos_enc.forward(EMB)
                print(POS)
                print(POS.shape)
#                 print(POS.size)
                print("Linear Layer Passthrough")
                QKV_attention, QK_soft = att.forward(POS)
                print(QKV_attention.shape)
                print(QKV_attention)
                QKV_FF = ffnn(QKV_attention)
                print("After feed forward")
                print(QKV_FF)
                print(QKV_FF.shape)
                
            
                first += 1

Linear(in_features=512, out_features=1536, bias=False)
Source sentence embedding
tensor([[[-23.3949, -31.2965,   2.6015,  ...,   8.6640,  18.8246, -30.0368],
         [ -9.3731, -33.2603,  29.3171,  ..., -27.7103,  16.1654,  20.7341],
         [-15.8117,  23.4906,  -8.3615,  ..., -16.2120, -36.1879, -31.1977],
         [  8.6436,   9.3361, -16.0813,  ..., -21.0107, -17.1292, -12.6608]]],
       grad_fn=<MulBackward0>)
Source sentence embedding plus positional encoding and dropout
tensor([[[-25.9943, -33.6627,   2.8905,  ...,  10.7377,  20.9162, -32.2631],
         [ -9.4796, -36.3556,  33.4877,  ..., -29.6781,  17.9616,  24.1490],
         [-16.5582,  25.6382,  -8.2501,  ..., -16.9022, -40.2086, -33.5530],
         [  9.7609,   9.2735, -17.5957,  ...,  -0.0000, -19.0322, -12.9565]]],
       grad_fn=<MulBackward0>)
torch.Size([1, 4, 512])
Linear Layer Passthrough
torch.Size([1, 4, 512])
tensor([[[-3.2665e+00,  1.4062e+01,  8.0322e-01,  ..., -6.2547e-28,
          -2.4050e-27, -5.0491e-2

In [ ]:
# Make the Encoder Layer (Self Attention and FFNN)

class EncoderLayer(nn.Module):
    def __init__(self, d_model, attn, ff, n_heads, dropout):
        super().__init__()
        self.attn = attn
        self.ff   = ff
        
        
        

In [ ]:
#jachiam


B = 5
L = 32
d = 16
w = 8

x = torch.as_tensor(np.random.rand(B,L,d), dtype=torch.float32)

class Attention(nn.Module):

    def __init__(self, d_in = 16, d_proj = 8, n_heads = 2, causal = True):  #ask josh why causal here  
        #a causal convolution has a window that overlaps only the current and previous timesteps
        #Causal convolutions are necessary because it would be cheating if the CNN was able to 
        #“see” information from the future timesteps that it is trying to predict.
        super().__init__()
        self.d_proj = d_proj
        self.n_heads = n_heads
        self.qkv_lin = nn.Linear(d_in, 3 * d_proj * n_heads, bias=False)
        self.causal = True

    def _mask(self, qk):
        mask = 1e9 * (torch.tril(torch.ones_like(qk)) - 1)
        return torch.tril(qk) + mask

    def forward(self, x):
        qkv_lin = self.qkv_lin(x).reshape(*x.shape[:2], self.n_heads, self.d_proj * 3) # B x L x n x 3d
        qkv_lin = qkv_lin.permute(0,2,1,3)                  # B x n x L x 3d 
        q, k, v = torch.split(qkv_lin, self.d_proj, dim=-1) # B x n x L x d
        q_pre = q.reshape(-1, *q.shape[2:])                 # Bn x L x d
        k_pre = k.reshape(-1, *k.shape[2:]).permute(0,2,1)  # Bn x d x L
        v_pre = v.reshape(-1, *v.shape[2:])                 # Bn x L x d
        qk = torch.bmm(q_pre,k_pre)                         # Bn x L x L
        qk_ = self._mask(qk) if self.causal else qk
        softmax_qk = torch.softmax(qk_, axis=-1)
        
   
        
        y = torch.bmm(softmax_qk, v_pre)                    # Bn x L x d
        y = y.reshape(x.shape[0], self.n_heads, x.shape[1], self.d_proj)  # B x n x L x d
        y = y.permute(0,2,1,3)   # B x L x n x d
        y = y.reshape(*x.shape[:2],-1)
        
        return y, softmax_qk.reshape(x.shape[0], self.n_heads, x.shape[1], x.shape[1])
    
        #q, k, v = torch.split(qkv_lin, self.d_proj, dim=2)
        #return q, k, v

torch.Size([64, 512])


In [ ]:



# # PE(pos, 2i)= sin(pos/((10000)^(2i/d_model)))
# # PE(pos, 2i+1)= cos(pos/((10000)^(2i/d_model)))

# d_model = 512
# a = torch.exp(torch.arange(0, d_model, 2) *  -(math.log(10000.0) / d_model))

# i=1
# aux = torch.zeros(256) + 10000.
# # print(aux)

# b = torch.pow(10000., -torch.arange(0, d_model, 2, dtype=torch.float)/d_model )

# print(a)
# print(b)


In [ ]:
# Dummy data
src_vocab_size = 20
trg_vocab_size = 20
src_token_ids_batch = torch.randint(1, 10, size=(3, 2))
trg_token_ids_batch = torch.randint(1, 10, size=(3, 2))



In [ ]:
# dropout_rate = 0.1
class LayerNorm(nn.Module):
    "Construct a layernorm module (See citation for details)."
    def __init__(self, features, eps=1e-6):
        super(LayerNorm, self).__init__()
        self.a_2 = nn.Parameter(torch.ones(features))
        self.b_2 = nn.Parameter(torch.zeros(features))
        self.eps = eps

    def forward(self, x):
        mean = x.mean(-1, keepdim=True)
        std = x.std(-1, keepdim=True)
        return self.a_2 * (x - mean) / (std + self.eps) + self.b_2

# Multiheaded attention
class SublayerConnection(nn.Module):
    """
    A residual connection followed by a layer norm.
    Note for code simplicity the norm is first as opposed to last.
    """
    def __init__(self, size, dropout):
        super(SublayerConnection, self).__init__()
        self.norm = LayerNorm(size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, sublayer):
        "Apply residual connection to any sublayer with the same size."
        return x + self.dropout(sublayer(self.norm(x)))


In [ ]:
# Encoder Implementation (In the AIAYN paper, we will stack 6 of these, may experiment with more)
# In each encoder layer, there is an attention mechanism and there a FFNN



# 1. Multihead Attention = Z

# 2. LayerNorm(Z + Drop(Z)) = Y

# 3. LayerNorm(Y + Drop(Fat-Relu(Y)))






In [ ]:
# Decoder Implementation

In [ ]:
def clones(module, N):
    "Produce N identical layers."
    return nn.ModuleList([copy.deepcopy(module) for _ in range(N)])


In [ ]:
class EncoderDecoder(nn.Module):
    """
    A standard Encoder-Decoder architecture. Base for this and many 
    other models.
    """
    def __init__(self, encoder, decoder, src_embed, tgt_embed, generator):
        super(EncoderDecoder, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.src_embed = src_embed
        self.tgt_embed = tgt_embed
        self.generator = generator
        
    def forward(self, src, tgt, src_mask, tgt_mask):
        "Take in and process masked src and target sequences."
        return self.decode(self.encode(src, src_mask), src_mask, tgt, tgt_mask)
    
    def encode(self, src, src_mask):
        return self.encoder(self.src_embed(src), src_mask)
    
    def decode(self, memory, src_mask, tgt, tgt_mask):
        return self.decoder(self.tgt_embed(tgt), memory, src_mask, tgt_mask)
    
    

class Encoder(nn.Module):
    "Core encoder is a stack of N layers"
    def __init__(self, layer, N):
        super(Encoder, self).__init__()
        self.layers = clones(layer, N)
        self.norm = LayerNorm(layer.size)
        
    def forward(self, x, mask):
        "Pass the input (and mask) through each layer in turn."
        for layer in self.layers:
            x = layer(x, mask)
        return self.norm(x)

    
class EncoderLayer(nn.Module):
    "Encoder is made up of self-attn and feed forward (defined below)"
    def __init__(self, size, self_attn, feed_forward, dropout):
        super(EncoderLayer, self).__init__()
        self.self_attn = self_attn
        self.feed_forward = feed_forward
        self.sublayer = clones(SublayerConnection(size, dropout), 2)
        self.size = size

    def forward(self, x, mask):
        "Follow Figure 1 (left) for connections."
        x = self.sublayer[0](x, lambda x: self.self_attn(x, x, x, mask))
        return self.sublayer[1](x, self.feed_forward)
    

In [ ]:
class Decoder(nn.Module):
    "Generic N layer decoder with masking."
    def __init__(self, layer, N):
        super(Decoder, self).__init__()
        self.layers = clones(layer, N)
        self.norm = LayerNorm(layer.size)
        
    def forward(self, x, memory, src_mask, tgt_mask):
        for layer in self.layers:
            x = layer(x, memory, src_mask, tgt_mask)
        return self.norm(x)
    
class DecoderLayer(nn.Module):
    "Decoder is made of self-attn, src-attn, and feed forward (defined below)"
    def __init__(self, size, self_attn, src_attn, feed_forward, dropout):
        super(DecoderLayer, self).__init__()
        self.size = size
        self.self_attn = self_attn
        self.src_attn = src_attn
        self.feed_forward = feed_forward
        self.sublayer = clones(SublayerConnection(size, dropout), 3)
 
    def forward(self, x, memory, src_mask, tgt_mask):
        "Follow Figure 1 (right) for connections."
        m = memory
        x = self.sublayer[0](x, lambda x: self.self_attn(x, x, x, tgt_mask))
        x = self.sublayer[1](x, lambda x: self.src_attn(x, m, m, src_mask))
        return self.sublayer[2](x, self.feed_forward)

In [ ]:
class Embeddings(nn.Module):
    def __init__(self, d_model, vocab):
        super(Embeddings, self).__init__()
        self.lut = nn.Embedding(vocab, d_model)
        self.d_model = d_model

    def forward(self, x):
        return self.lut(x) * math.sqrt(self.d_model)

In [ ]:
class Generator(nn.Module):
    "Define standard linear + softmax generation step."
    def __init__(self, d_model, vocab):
        super(Generator, self).__init__()
        self.proj = nn.Linear(d_model, vocab)

    def forward(self, x):
        return F.log_softmax(self.proj(x), dim=-1)

In [ ]:
def make_model(src_vocab, tgt_vocab, N=6, d_model=512, d_ff=2048, h=8, dropout=0.1):
    "Helper: Construct a model from hyperparameters."
    c = copy.deepcopy
    attn = Attention2(n_heads=h, d_model=d_model, dropout=dropout)
    ff = FFNN(d_in=d_model, d_within=d_ff, dropout=dropout)
#     position = PositionalEncoding(d_model, dropout)
    position = PositionalEncoding(d_model = 512, dropout=0.1, max_len=5000)
    model = EncoderDecoder(
        Encoder(EncoderLayer(d_model, c(attn), c(ff), dropout), N),
        Decoder(DecoderLayer(d_model, c(attn), c(attn), 
                             c(ff), dropout), N),
        nn.Sequential(Embeddings(d_model, src_vocab), c(position)),
        nn.Sequential(Embeddings(d_model, tgt_vocab), c(position)),
        Generator(d_model, tgt_vocab))
    
    # This was important from their code.  # Initialize parameters with Glorot / fan_avg.
    for p in model.parameters():
        if p.dim() > 1:
            nn.init.xavier_uniform(p)
    return model

In [ ]:
tmp_model = make_model(10, 10, 2)
None

#TODO: change Embeddings to InputEmbeddings

<ipython-input-336-5e1617b601b6>:20: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  nn.init.xavier_uniform(p)


In [ ]:
 model = make_model(len(SRC.vocab), len(TGT.vocab), N=6)
    


<ipython-input-336-5e1617b601b6>:20: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  nn.init.xavier_uniform(p)


In [ ]:
x = torch.tensor([1, 2, 3, 4])
a= torch.unsqueeze(x, 0)
b= torch.unsqueeze(x, 1)
# c= torch.unsqueeze(x, 2)
d= torch.unsqueeze(x, -1)

print(x)
print(a)
print(b)
print(d)

In [ ]:

# def get_inter(pos, i, d_model):
#     x = (1 / np.power(10000., (2 * (i//2)) / np.float32(d_model)))
#     return pos*x

# def pos_encode(position, d_model):
#     self.wave_pos = get_inter(torch.arange(position).unsqueeze(1), torch.arange(d_model).unsqueeze(1), d_model = 512)
        
#     # apply sin to even indices in the array; 2i
#     wave_pos[:, 0::2] = np.sin(wave_pos[:, 0::2])
    
#     # apply cos to odd indices in the array; 2i+1
#     wave_pos[:, 1::2] = np.cos(wave_pos[:, 1::2])
        
        
# def get_angles(pos, i, d_model):
#     angle_rates = 1 / np.power(10000, (2 * (i//2)) / np.float32(d_model))
#     return pos * angle_rates

# def positional_encoding(position, d_model):
#     angle_rads = get_angles(np.arange(position)[:, np.newaxis],     np.arange(d_model)[np.newaxis, :],                 d_model)
  
#   # apply sin to even indices in the array; 2i
#     angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])
  
#   # apply cos to odd indices in the array; 2i+1
#     angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])
    
#     pos_encoding = angle_rads[np.newaxis, ...]
    
#     return pos_encoding

# pos_encoding = positional_encoding(tokens, dimensions)
# pos_enc = pos_encode(tokens, dimensions)
# print (pos_encoding.shape)
# # print(pos_encoding)

# print(np.arange(10))
# print(np.arange(10)[:, np.newaxis])

# print(torch.arange(10))
# print(torch.arange(10).unsqueeze(1))

# # print(np.arange())